In [ ]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from dateutil import tz

import pynwb
import numpy as np

# pip install python-intervals, *NOT* pip install intervals
import intervals as iv

import query_helpers as qu
import nspike_helpers as ns

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker


plt.rcParams.update({'font.size': 14})
plt.rcParams.update({'lines.solid_capstyle': 'butt'})
mdates.rcParams.update({'date.autoformatter.microsecond': '%H:%M:%S.%f'})

In [ ]:
anim = 'Bon'
file1 = './Bon03_test.nwb'
file2 = './Bon04_test.nwb'
multiday_filename = './Bon_multiday_test.nwb'
multiday_linkfalse_filename = './Bon_multiday_test_linkfalse.nwb'

### Open 2 existing nwb files, and create master file

In [ ]:
manager = pynwb.get_manager()

io1 = pynwb.NWBHDF5IO(file1, mode='r', manager=manager)
nwbf1 = io1.read()

io2 = pynwb.NWBHDF5IO(file2, mode='r', manager=manager)
nwbf2 = io2.read()

# # Session-specific params
# data_dir = '/opt/data46/FrankData/kkay/Bon/'
# #data_dir = '/Users/tjd/Data/FrankData/kkay/Bon'
# data_source = 'Animal Bond'
# anim = 'Bon' 
# day = 3

# 'Wall clock' (i.e. actual) date and time of the Nspike time = 0 for this experiment.
# NOTE: this is not the true zero_time, as we don't have easy access to that without digging in lab notebooks.
# TODO: dig in lab notebooks, at least to get the date of recording.
dataset_zero_time = datetime(2006, 1, 1, 12, 0, 0, tzinfo=tz.gettz('US/Pacific'))

# General params/presets
file_create_date = datetime.now(tz.tzlocal())

source = 'NSpike data acquisition system'

nwb_filename = multiday_filename

nwbf_multi = pynwb.NWBFile(
                   source=source,
                   session_description='Converted NSpike data',
                   identifier=anim,
                   session_start_time=dataset_zero_time,
                   file_create_date=file_create_date,
                   lab='Frank Laboratory',
                   experimenter='Mattias Karlsson',
                   institution='UCSF',
                   experiment_description='Multiple days of recordings from awake behaving rat')

#### Link to some data from each source file

In [ ]:
es1 = nwbf1.get_acquisition('LFP').electrical_series['boneeg-3-12']
es2 = nwbf2.get_acquisition('LFP').electrical_series['boneeg-4-12']

In [ ]:
nwbf_multi.add_acquisition(es1)
nwbf_multi.add_acquisition(es2)

#### Write out master file
Must use same 'manager' as used on read, above

In [ ]:
io_multi = pynwb.NWBHDF5IO(multiday_filename, mode='w', manager=manager)
io_multi.write(nwbf_multi, link_data=True)
io_multi.close()

In [ ]:
## link_data=False has no effect here (vs. link_data=true)
# io_multi = pynwb.NWBHDF5IO(multiday_linkfalse_filename, mode='w', manager=manager)
# io_multi.write(nwbf_multi, link_data=False)
# io_multi.close()

In [ ]:
io1.close()
io2.close()

### Load new 'master' file and check whether we can access data

In [ ]:
io_multi_read = pynwb.NWBHDF5IO(multiday_filename, mode='r')
nwbf_multi_r = io_multi_read.read()

In [ ]:
print(nwbf_multi_r.acquisition['boneeg-3-12'].data[-10:-1])
print(nwbf_multi_r.acquisition['boneeg-4-12'].data[-10:-1])

### Copy master file, dereferencing links
File goes from 17KB -> 240MB

In [ ]:
io_multi_read.copy_file(multiday_filename, './h5copy_test.nwb')